In [18]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, count, demographic_parity_difference
from ipynb.fs.defs import ERED

### Load Data

In [19]:
# Overall data
# Sentence-transformers test set
# train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))    # Original data
train_data = pd.read_pickle(open("sentence-transformers/clean_train_emb.pkl", "rb"))  # Noice reduced
# dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))  # Original data
dev_data = pd.read_pickle(open("sentence-transformers/clean_dev_emb.pkl", "rb"))  # Noice reduced

# TFIDF test set
# train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
# dev_data = pd.read_pickle(open("tfidf/dev_tfidf.pkl", "rb"))

train_X = list(train_data['TFIDF'])
train_y = (train_data['Sentiment'] == 'positive') * 1

dev_X = list(dev_data['TFIDF'])
dev_y  = (dev_data['Sentiment'] == 'positive') * 1

# AAE
# Filter and extract all AAE data
AAE_train_data = train_data[train_data['Demographic'] == 'AAE']
AAE_train_X = list(AAE_train_data['TFIDF'])
AAE_train_y = (AAE_train_data['Sentiment'] == 'positive') * 1

AAE_dev_data = dev_data[dev_data['Demographic'] == 'AAE']
AAE_dev_X = list(AAE_dev_data['TFIDF'])
AAE_dev_y  = (AAE_dev_data['Sentiment'] == 'positive') * 1

# SAE
# Filter and extract all SAE data
SAE_train_data = train_data[train_data['Demographic'] == 'SAE']
SAE_train_X = list(SAE_train_data['TFIDF'])
SAE_train_y = (SAE_train_data['Sentiment'] == 'positive') * 1

SAE_dev_data = dev_data[dev_data['Demographic'] == 'SAE']
SAE_dev_X = list(SAE_dev_data['TFIDF'])
SAE_dev_y  = (SAE_dev_data['Sentiment'] == 'positive') * 1

### Train Model

In [20]:
nb = GaussianNB()
nb.fit(train_X, train_y)

GaussianNB()

### Prediction

#### 1. Overall

In [21]:
overall_labels_predict = nb.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.69      0.62      0.65      2000
           1       0.65      0.72      0.68      2000

    accuracy                           0.67      4000
   macro avg       0.67      0.67      0.67      4000
weighted avg       0.67      0.67      0.67      4000

[[1244  756]
 [ 568 1432]]


#### 2. AAE

In [22]:
AAE_labels_predict = nb.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.69      0.44      0.54      1000
           1       0.59      0.80      0.68      1000

    accuracy                           0.62      2000
   macro avg       0.64      0.62      0.61      2000
weighted avg       0.64      0.62      0.61      2000

[[439 561]
 [201 799]]


#### 3. SAE

In [23]:
SAE_labels_predict = nb.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.69      0.81      0.74      1000
           1       0.76      0.63      0.69      1000

    accuracy                           0.72      2000
   macro avg       0.73      0.72      0.72      2000
weighted avg       0.73      0.72      0.72      2000

[[805 195]
 [367 633]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [24]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.669
count        4000
dtype: object
            accuracy count
Demographic               
AAE            0.619  2000
SAE            0.719  2000


#### 2. Selection rate + Demographic parity difference

In [25]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.547
Demographic
AAE     0.68
SAE    0.414
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.266


#### 3. Error Rate Equality Difference

In [26]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.366
Sum of FNR difference: 0.166
Sum of Error Rate Equality Difference: 0.532


0.532

---

## Bias Mitigation

### Exponentiated Gradient

In [27]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
np.random.seed(0)

In [28]:
constraint = DemographicParity()
classifier = GaussianNB()
mitigator = ExponentiatedGradient(classifier, constraints=constraint)
mitigator.fit(np.array(train_X), train_y, sensitive_features=train_data['Demographic'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x1428aabc0>,
                      estimator=GaussianNB(), nu=0.00118112205708506)

### Prediction

#### 1. Overall

In [29]:
overall_labels_predict = mitigator.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.67      0.65      0.66      2000
           1       0.66      0.67      0.67      2000

    accuracy                           0.66      4000
   macro avg       0.66      0.66      0.66      4000
weighted avg       0.66      0.66      0.66      4000

[[1296  704]
 [ 652 1348]]


#### 2. AAE

In [30]:
AAE_labels_predict = mitigator.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.61      0.59      0.60      1000
           1       0.60      0.63      0.62      1000

    accuracy                           0.61      2000
   macro avg       0.61      0.61      0.61      2000
weighted avg       0.61      0.61      0.61      2000

[[591 409]
 [374 626]]


#### 3. SAE

In [31]:
SAE_labels_predict = mitigator.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.72      0.71      0.71      1000
           1       0.71      0.72      0.72      1000

    accuracy                           0.71      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.72      0.71      0.71      2000

[[707 293]
 [277 723]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [32]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.661
count        4000
dtype: object
            accuracy count
Demographic               
AAE            0.604  2000
SAE            0.718  2000


#### 2. Selection rate + Demographic parity difference

In [33]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.513
Demographic
AAE    0.521
SAE    0.505
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.016


#### 3. Error Rate Equality Difference

In [34]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.116
Sum of FNR difference: 0.097
Sum of Error Rate Equality Difference: 0.213


0.21299999999999997